# Delta Lake Lab 
## Unit 4: CRUD Support

This lab is powered by Dataproc Serverless Spark.

In the previous unit, we -
1. Create an unpartitioned delta table
2. Created a partitioned delta table called loan_db.loans_by_state_delta
3. Studied the files created & layout in the datalake
4. Learned how to look at delta table details
5. Looked at history (there was not any)
6. Created a manifest file
7. Reviewed entries in the Hive metastore

In this unit, we will learn how to -
1. Delete a record and study the delta log
2. Insert a record and study the delta log
3. Update a record and study the delta log
4. Upsert and study the delta log

### 1. Imports

In [1]:
import pandas as pd

from pyspark.sql.functions import month, date_format
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession

from delta.tables import *

import warnings
warnings.filterwarnings('ignore')

### 2. Create a Spark session powered by Cloud Dataproc 

In [2]:
spark = SparkSession.builder.appName('Loan Analysis').getOrCreate()
spark

22/10/22 18:57:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### 3. Declare variables

In [3]:
project_id_output = !gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID = project_id_output[0]
print("PROJECT_ID: ", PROJECT_ID)

PROJECT_ID:  delta-lake-lab


In [4]:
project_name_output = !gcloud projects describe $PROJECT_ID | grep name | cut -d':' -f2 | xargs
PROJECT_NAME = project_name_output[0]
print("PROJECT_NAME: ", PROJECT_NAME)

PROJECT_NAME:  delta-lake-lab


In [5]:
project_number_output = !gcloud projects describe $PROJECT_ID | grep projectNumber | cut -d':' -f2 | xargs
PROJECT_NUMBER = project_number_output[0]
print("PROJECT_NUMBER: ", PROJECT_NUMBER)

PROJECT_NUMBER:  885979867746


In [6]:
DATA_LAKE_ROOT_PATH= f"gs://dll-data-bucket-{PROJECT_NUMBER}"
DELTA_LAKE_DIR_ROOT = f"{DATA_LAKE_ROOT_PATH}/delta-consumable"

In [7]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-48902d8f-0572-4735-b0d7-95b1927cb294-c000.snappy.parquet

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json


### 4. Delete support

In [8]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta WHERE addr_state='IA'").show()

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/spark/conf/ivysettings.xml will be used


+----------+-----+
|addr_state|count|
+----------+-----+
|        IA|    1|
+----------+-----+



In [11]:
spark.sql("DELETE FROM loan_db.loans_by_state_delta WHERE addr_state='IA'")

DataFrame[num_affected_rows: bigint]

In [12]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta WHERE addr_state='IA'").show()

+----------+-----+
|addr_state|count|
+----------+-----+
+----------+-----+



Lets look at the data lake:

In [14]:
# Note how the deleted created a json in the delta log directory
!gsutil ls -r $DELTA_LAKE_DIR_ROOT 

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-3eb98e04-4353-4e5f-a8a2-17a570111981-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-48902d8f-0572-4735-b0d7-95b1927cb294-c000.snappy.parquet

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000001.json


Lets look at the delta log:

In [15]:
# This is the original log
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000000.json 

{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"a67aa212-a88d-4e69-93fb-344e20b071e8","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"addr_state\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"count\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1666463629448}}
{"add":{"path":"part-00000-48902d8f-0572-4735-b0d7-95b1927cb294-c000.snappy.parquet","partitionValues":{},"size":978,"modificationTime":1666463636970,"dataChange":true,"stats":"{\"numRecords\":51,\"minValues\":{\"addr_state\":\"AK\",\"count\":1},\"maxValues\":{\"addr_state\":\"WY\",\"count\":1},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"commitInfo":{"timestamp":1666463640978,"operation":"WRITE","operationParameters":{"mode":"Overwrite","partitionBy":"[]"},"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numFiles":"1","numOutp

In [16]:
# Note the delete in this log
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000001.json 

{"remove":{"path":"part-00000-48902d8f-0572-4735-b0d7-95b1927cb294-c000.snappy.parquet","deletionTimestamp":1666465108554,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":978}}
{"add":{"path":"part-00000-3eb98e04-4353-4e5f-a8a2-17a570111981-c000.snappy.parquet","partitionValues":{},"size":973,"modificationTime":1666465108349,"dataChange":true,"stats":"{\"numRecords\":50,\"minValues\":{\"addr_state\":\"AK\",\"count\":1},\"maxValues\":{\"addr_state\":\"WY\",\"count\":1},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"commitInfo":{"timestamp":1666465108571,"operation":"DELETE","operationParameters":{"predicate":"[\"(spark_catalog.loan_db.loans_by_state_delta.addr_state = 'IA')\"]"},"readVersion":0,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numRemovedFiles":"1","numCopiedRows":"50","numAddedChangeFiles":"0","executionTimeMs":"3377","numAddedFiles":"1","rewriteTimeMs":"1549","numDeletedRows":"1","scanTimeMs":"1827"},"engineInfo":"A

### 5. Create (Insert) support

In [17]:
spark.sql("INSERT INTO loan_db.loans_by_state_delta VALUES ('IA',222222)")

DataFrame[]

In [18]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta WHERE addr_state='IA'").show()

+----------+------+
|addr_state| count|
+----------+------+
|        IA|222222|
+----------+------+



In [19]:
# Note how the insert created a new parquet file and in the delta log, yet another json
!gsutil ls -r $DELTA_LAKE_DIR_ROOT 

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-3eb98e04-4353-4e5f-a8a2-17a570111981-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-48902d8f-0572-4735-b0d7-95b1927cb294-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-c36f0527-e995-41d2-a8ea-776cc865f816-c000.snappy.parquet

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000001.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000002.json


In [20]:
# Lets check for the insert
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000002.json 

{"add":{"path":"part-00000-c36f0527-e995-41d2-a8ea-776cc865f816-c000.snappy.parquet","partitionValues":{},"size":725,"modificationTime":1666465229119,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"IA\",\"count\":222222},\"maxValues\":{\"addr_state\":\"IA\",\"count\":222222},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"commitInfo":{"timestamp":1666465229221,"operation":"WRITE","operationParameters":{"mode":"Append","partitionBy":"[]"},"readVersion":1,"isolationLevel":"Serializable","isBlindAppend":true,"operationMetrics":{"numFiles":"1","numOutputRows":"1","numOutputBytes":"725"},"engineInfo":"Apache-Spark/3.3.0 Delta-Lake/2.1.0","txnId":"3371a03f-bbf1-4d59-b887-c0a03764d4d6"}}


### 6. Update support

Lets update a record & see the changes in the delta log directory

In [21]:
spark.sql("UPDATE loan_db.loans_by_state_delta SET count = 11111 WHERE addr_state='IA'")

DataFrame[num_affected_rows: bigint]

In [22]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta WHERE addr_state='IA'").show()

+----------+-----+
|addr_state|count|
+----------+-----+
|        IA|11111|
+----------+-----+



In [23]:
# Note how the update created a new parquet file and in the delta log, yet another json
!gsutil ls -r $DELTA_LAKE_DIR_ROOT 

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-228eb829-1144-4a6d-a0e2-4fd39d9e9f57-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-3eb98e04-4353-4e5f-a8a2-17a570111981-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-48902d8f-0572-4735-b0d7-95b1927cb294-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-c36f0527-e995-41d2-a8ea-776cc865f816-c000.snappy.parquet

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000001.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000002.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000

In [24]:
# Lets check for the update
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000003.json 

{"remove":{"path":"part-00000-c36f0527-e995-41d2-a8ea-776cc865f816-c000.snappy.parquet","deletionTimestamp":1666465437187,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":725}}
{"add":{"path":"part-00000-228eb829-1144-4a6d-a0e2-4fd39d9e9f57-c000.snappy.parquet","partitionValues":{},"size":725,"modificationTime":1666465437069,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"IA\",\"count\":11111},\"maxValues\":{\"addr_state\":\"IA\",\"count\":11111},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"commitInfo":{"timestamp":1666465437189,"operation":"UPDATE","operationParameters":{"predicate":"(addr_state#1557 = IA)"},"readVersion":2,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numRemovedFiles":"1","numCopiedRows":"0","numAddedChangeFiles":"0","executionTimeMs":"2121","scanTimeMs":"1040","numAddedFiles":"1","numUpdatedRows":"1","rewriteTimeMs":"1079"},"engineInfo":"Apache-Spark/3.3.0 Delta-Lake/2.1.0","txn

### 7. Upsert support

In [25]:
toBeMergedRows = [('IA', 555), ('CA', 12345), ('IN', 6666)]
toBeMergedColumns = ['addr_state', 'count']
toBeMergedDF = spark.createDataFrame(toBeMergedRows, toBeMergedColumns)
toBeMergedDF.createOrReplaceTempView("to_be_merged_table")
toBeMergedDF.orderBy("addr_state").show(3)

+----------+-----+
|addr_state|count|
+----------+-----+
|        CA|12345|
|        IA|  555|
|        IN| 6666|
+----------+-----+



In [26]:
spark.sql("DELETE FROM loan_db.loans_by_state_delta WHERE addr_state='IA'")

DataFrame[num_affected_rows: bigint]

In [27]:
spark.sql("SELECT addr_state,count FROM loan_db.loans_by_state_delta WHERE addr_state in ('IA','CA','IN') ORDER BY addr_state").show()

+----------+-----+
|addr_state|count|
+----------+-----+
|        CA|    1|
|        IN|    1|
+----------+-----+



In [28]:
mergeSQLStatement = "MERGE INTO loan_db.loans_by_state_delta as d USING to_be_merged_table as m ON (d.addr_state = m.addr_state) WHEN MATCHED THEN UPDATE SET * WHEN NOT MATCHED THEN INSERT * "

print(mergeSQLStatement)


MERGE INTO loan_db.loans_by_state_delta as d USING to_be_merged_table as m ON (d.addr_state = m.addr_state) WHEN MATCHED THEN UPDATE SET * WHEN NOT MATCHED THEN INSERT * 


In [29]:
spark.sql(mergeSQLStatement)

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [30]:
spark.sql("SELECT addr_state,count FROM loan_db.loans_by_state_delta WHERE addr_state in ('IA','CA','IN') ORDER BY addr_state").show()

+----------+-----+
|addr_state|count|
+----------+-----+
|        CA|12345|
|        IA|  555|
|        IN| 6666|
+----------+-----+



In [31]:
# Note how the update created a new parquet file and in the delta log, yet another json
!gsutil ls -r $DELTA_LAKE_DIR_ROOT 

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-228eb829-1144-4a6d-a0e2-4fd39d9e9f57-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-3eb98e04-4353-4e5f-a8a2-17a570111981-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-48902d8f-0572-4735-b0d7-95b1927cb294-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-61eebddf-adb1-4088-9520-6a953e6d3fff-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-7b773887-eac1-481b-aa71-e57bd469f977-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-c36f0527-e995-41d2-a8ea-776cc865f816-c000.snappy.parquet

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-buc

In [32]:
# Lets check for the delete
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000004.json 

{"remove":{"path":"part-00000-228eb829-1144-4a6d-a0e2-4fd39d9e9f57-c000.snappy.parquet","deletionTimestamp":1666465582736,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":725}}
{"add":{"path":"part-00000-7b773887-eac1-481b-aa71-e57bd469f977-c000.snappy.parquet","partitionValues":{},"size":397,"modificationTime":1666465582663,"dataChange":true,"stats":"{\"numRecords\":0,\"minValues\":{},\"maxValues\":{},\"nullCount\":{}}"}}
{"commitInfo":{"timestamp":1666465582737,"operation":"DELETE","operationParameters":{"predicate":"[\"(spark_catalog.loan_db.loans_by_state_delta.addr_state = 'IA')\"]"},"readVersion":3,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numRemovedFiles":"1","numCopiedRows":"0","numAddedChangeFiles":"0","executionTimeMs":"1365","numAddedFiles":"1","rewriteTimeMs":"625","numDeletedRows":"1","scanTimeMs":"740"},"engineInfo":"Apache-Spark/3.3.0 Delta-Lake/2.1.0","txnId":"142efbb0-5d39-459e-ad9b-b1af031229d7"}}


In [33]:
# Lets check for the upsert
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000005.json 

{"remove":{"path":"part-00000-3eb98e04-4353-4e5f-a8a2-17a570111981-c000.snappy.parquet","deletionTimestamp":1666465614163,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":973}}
{"add":{"path":"part-00000-61eebddf-adb1-4088-9520-6a953e6d3fff-c000.snappy.parquet","partitionValues":{},"size":993,"modificationTime":1666465614080,"dataChange":true,"stats":"{\"numRecords\":51,\"minValues\":{\"addr_state\":\"AK\",\"count\":1},\"maxValues\":{\"addr_state\":\"WY\",\"count\":12345},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"commitInfo":{"timestamp":1666465614174,"operation":"MERGE","operationParameters":{"predicate":"(d.addr_state = m.addr_state)","matchedPredicates":"[{\"actionType\":\"update\"}]","notMatchedPredicates":"[{\"actionType\":\"insert\"}]"},"readVersion":4,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numTargetRowsCopied":"48","numTargetRowsDeleted":"0","numTargetFilesAdded":"1","scanTimeMs":"2119","numTargetRowsUpdated":

### THIS CONCLUDES THIS UNIT. PROCEED TO THE NEXT NOTEBOOK